In [1]:
# Setup
!pip install -q -U google-genai pandas weave pydantic nest_asyncio rich

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.1/719.1 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.3/813.3 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
# Módulos

import os
import asyncio
import pandas as pd
import numpy as np
from google import genai
import weave
from pydantic import BaseModel, Field
import json
from google.colab import userdata
import nest_asyncio
import sys
import io
import contextlib
import builtins # Para mockear input
from datetime import datetime
from typing import List, Optional

# --- LIBRERÍA RICH (UI) ---
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.syntax import Syntax
from rich.rule import Rule
from rich.theme import Theme




In [11]:
# --- CONFIGURACIÓN API Key ---
# En un entorno real, usar os.environ.get("GOOGLE_API_KEY")
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEYL')

client = genai.Client(api_key=GOOGLE_API_KEY)

# 🌟 Cortex v3.0: Framework Agéntico con UX/UI Profesional

Esta es la versión definitiva del **Cortex Agentic Framework**. Representa la culminación de los 5 niveles de desarrollo, integrando lógica compleja, ciencia de datos, tutoría de código y una experiencia de usuario (DX/UX) de alto nivel.

## 🚀 Características Principales

### 1. Arquitectura Híbrida "Multi-Tool"
El agente posee un cerebro central (Gemini 2.5) que orquesta el uso de herramientas especializadas según la intención del usuario:
* **🐼 Pandas Analysis:** Para ciencia de datos segura sobre dataframes en memoria.
* **🐍 Python Tutor:** Un entorno de ejecución (Sandbox) capaz de escribir, ejecutar y verificar ejercicios de programación, incluyendo simulación de inputs de usuario.

### 2. Observabilidad y Auditoría (Enterprise Ready)
* **Weave (WandB):** Traza cada llamada al LLM, permitiendo depurar costos, latencia y alucinaciones en una interfaz web.
* **Sistema de Logging (`agent_log.txt`):** Genera un registro persistente y detallado con timestamps de cada pensamiento, acción y error del agente. "Si no está en los logs, no ocurrió".

### 3. Experiencia de Usuario (Rich UI)
Implementamos la librería **Rich** para transformar la salida de la consola en una interfaz visual atractiva:
* **🧠 Pensamientos:** Paneles azules para el razonamiento interno.
* **🛠️ Herramientas:** Resaltado de sintaxis (Syntax Highlighting) para el código Python generado.
* **🟢 Respuesta Final:** Renderizado de Markdown (negritas, listas, bloques de código) en paneles verdes.
* **🔴 Manejo de Errores:** Alertas visuales claras para excepciones críticas (como Cuota Excedida).

### 4. Robustez Técnica
* **Input Mocking:** El agente puede probar código interactivo (que usa `input()`) sin bloquear la ejecución, simulando entradas de teclado.
* **Rate Limiting:** Control inteligente de la velocidad de peticiones para respetar la cuota de la API gratuita.
* **Ciclo ReAct Anti-Loop:** Reglas estrictas en el Prompt del Sistema para evitar que el agente repita tareas innecesariamente.

---
**Instrucciones de Ejecución:**
1.  Asegúrate de tener instaladas las dependencias: `pip install rich weave google-genai pandas pydantic nest_asyncio`.
2.  Configura tu `API_KEY` de Google Gemini.
3.  Al ejecutar, verás la salida coloreada en consola y se generará el archivo `agent_log.txt` en el directorio actual.

In [12]:
# Dataset - 1000 filas

# 1. Cargar el dataset
df = pd.read_csv('dataset_notas.csv')

df.head()

,gender,math score,reading score,writing score,attendance
0,female,72,72,74,0.19
1,female,69,90,88,0.39
2,female,90,95,93,0.54
3,male,47,57,44,0.17
4,male,76,78,75,0.56


In [13]:
# se genera un resumen estadístico descriptivo del dataset
df.describe(include='all')

,gender,math score,reading score,writing score,attendance
count,1000,1000.00000,1000.000000,1000.000000,1000.000000
unique,2,NaN,NaN,NaN,NaN
top,female,NaN,NaN,NaN,NaN
freq,518,NaN,NaN,NaN,NaN
mean,NaN,66.08900,69.169000,68.054000,0.560430
std,NaN,15.16308,14.600192,15.195657,0.256027
min,NaN,0.00000,17.000000,10.000000,0.100000
25%,NaN,57.00000,59.000000,57.750000,0.340000
50%,NaN,66.00000,70.000000,69.000000,0.560000
75%,NaN,77.00000,79.000000,79.000000,0.780000


In [17]:
# En estas celdas se evalua directamente lo que deberia
# retornar el agente cuando utiliza la herramienta pandas_analysis

# se calcula la media de math score
df['math score'].mean()

np.float64(66.089)

In [18]:
# se calcula la mediana de math score
df['math score'].median()

66.0

In [19]:
# se calcula la moda de math score
df['math score'].mode()

,math score
0,65


In [20]:
# se cuenta cuantas personas tienen asistencia perfecta
(df['attendance'] == 1.0).sum()

np.int64(6)

In [21]:
# Parche para Colab
nest_asyncio.apply()

# Inicializamos Weave para ver todo lo que pasa
weave.init("cortex-agent-multitool")

# Configuración de Rich (Colores personalizados)
custom_theme = Theme({
    "info": "dim cyan",
    "warning": "magenta",
    "danger": "bold red"
})
console = Console(theme=custom_theme)

# --- 0. CLASE LOGGER (AUDITORÍA) ---
class CortexLogger:
    def __init__(self, filename="agent_log.txt"):
        self.filename = filename
        # Crear archivo si no existe o añadir separador de nueva sesión
        with open(self.filename, "a", encoding="utf-8") as f:
            f.write(f"\n{'='*50}\nSESIÓN INICIADA: {datetime.now()}\n{'='*50}\n")

    def log(self, step_type, content):
        """Guarda eventos en el archivo de texto con timestamp."""
        timestamp = datetime.now().strftime("%H:%M:%S")
        entry = f"[{timestamp}] [{step_type.upper()}]\n{content}\n{'-'*30}\n"

        with open(self.filename, "a", encoding="utf-8") as f:
            f.write(entry)

# --- 1. MODELOS DE DATOS (Pydantic) ---
# Estructuramos la respuesta del LLM para que no hable texto libre, sino JSON
class AgentAction(BaseModel):
    thought: str = Field(description="Razonamiento interno sobre el problema y qué herramienta usar. Si ya ejecutaste código y funcionó, ve a final_answer.")
    tool: str = Field(description="Herramienta a usar: 'pandas_analysis', 'python_tutor_exec' o 'final_answer'")
    tool_input: str = Field(description="Código python a ejecutar o respuesta final")

    # ÚLTIMO CAMBIO: Campo opcional para que el Agente defina sus propios inputs de prueba
    mock_inputs: Optional[List[str]] = Field(
        default=None,
        description="Lista de inputs simulados (strings) SOLO si usas 'python_tutor_exec' y el código requiere input(). Ej: ['Juan', '50', 'fin']"
    )

# --- 2. HERRAMIENTAS (Tools) ---
class CortexTools:
    def __init__(self, df):
        self.df = df

    @contextlib.contextmanager
    def capture_output_and_mock_input(self, inputs=None):
        """
        Captura stdout y simula stdin (input).
        Si el código pide input(), se le entregarán los valores de la lista 'inputs'.
        """
        new_out = io.StringIO()
        old_out = sys.stdout
        original_input = builtins.input

        # Generador de inputs simulados
        input_iter = iter(inputs or [])

        def mock_input(prompt=""):
            # Mostramos el prompt del input en la salida capturada para realismo
            print(prompt, end="")
            try:
                value = next(input_iter)
                print(value) # Escribimos el valor "tecleado" en la salida
                return value
            except StopIteration:
                # Retorna vacío para no romper, aunque el código debería manejarlo
                print("[Auto-Fin de Inputs]")
                return ""

        try:
            sys.stdout = new_out
            builtins.input = mock_input # Reemplazamos el input real
            yield new_out
        finally:
            sys.stdout = old_out
            builtins.input = original_input # Restauramos el input real

    @weave.op()
    def pandas_analysis(self, query_code: str) -> str:
        """
        Herramienta 1:
        Data Science (Usa eval() porque esperamos un retorno de valor).
        Ejecuta código de pandas generado por el LLM.
        """
        try:
            # ADVERTENCIA DE SEGURIDAD: eval() es peligroso en prod.
            # Aquí lo usamos restringido al dataframe 'df' para fines educativos.
            local_vars = {"df": self.df}
            result = eval(query_code, {}, local_vars)

            # --- MEJORA PARA ESTADÍSTICA ---
            # Si el resultado es una Serie (común en .mode()), la convertimos a lista
            if isinstance(result, pd.Series):
                return f"Resultado (Serie/Lista): {result.tolist()}"
            elif isinstance(result, pd.DataFrame):
                return f"Resultado (Tabla):\n{result.to_markdown()}"

            return str(result)
        except Exception as e:
            return f"Error ejecutando pandas: {e}"

    @weave.op()
    def python_tutor_exec(self, code_str: str, mock_inputs: List[str] = None) -> str:
        """
        Herramienta 2:
        Tutoría Python (Usa exec() para soportar bucles y funciones).
        Captura lo que el código imprime.
        """
        try:
            # Entorno aislado para ejecutar el ejercicio
            local_env = {}

            # Usamos los inputs provistos por el Agente o una lista vacía por defecto
            inputs_to_use = mock_inputs if mock_inputs else []

            # Definimos unos inputs de prueba genéricos por si el Agente olvida definirlos
            # (El agente debería ser capaz de auto-testearse, pero esto es un fallback)
            #test_inputs = ["Ana", "Pedro", "Maria", "Leandro", "Carla", "fin"]

            # Ejecutamos capturando el print()
            with self.capture_output_and_mock_input(inputs_to_use) as captured_io:
                exec(code_str, {}, local_env)

            output = captured_io.getvalue()

            if not output:
                return "⚠️ El código se ejecutó sin errores, pero no imprimió nada. Recuerda usar print() para ver el resultado."

            return f"✅ Ejecución Exitosa.\nSalida en consola:\n{output}"

        except Exception as e:
            return f"❌ Error de Ejecución:\n{e}"

# --- 3. EL CEREBRO (LLM) ---
class CortexBrain:
    def __init__(self):
        self.model = "gemini-2.5-flash"

    @weave.op()
    async def decide(self, history: str, schema: str) -> AgentAction:
        prompt = f"""

        Eres Cortex, un asistente híbrido: Data Scientist y Tutor de Python.

        TUS HERRAMIENTAS:

        1. [pandas_analysis]:
           - SOLO para preguntas sobre el dataset de alumnos cargado en memoria.
           - El dataframe tiene columnas como 'gender', 'math score', 'reading score', 'writing score', 'attendance'.
           - Usa sintaxis de una línea (eval). Ej: "df['math score'].mean()"

           EJEMPLOS DE CÓDIGO A GENERAR (como una única expresión para 'tool_input'):
           - Media (Promedio): "df['math score'].mean()"
           - Mediana (Valor central): "df['math score'].median()"
           - Moda (Valor más frecuente): "df['math score'].mode()"
           - Conteo de asistencia perfecta: "(df['attendance'] == 1.0).sum()"
           - Describe completo: "df.describe()"

        2. [python_tutor_exec]:
           - ÚSALA para resolver ejercicios de programación generales.
           - Genera un script completo (imports, bucles, funciones, etc).
           - IMPORTANTE: El código DEBE tener 'print()' para mostrar el resultado.
           - NOTA: El entorno de ejecución es NO INTERACTIVO.
           - Si tu código usa 'input()', DEBES proveer una lista de valores de prueba en el campo 'mock_inputs'.
           - Ejemplo: Si pides nombres hasta 'fin', mock_inputs=["Ana", "Luis", "fin"].
           - Sé creativo con los datos de prueba.
           - Preocúpate siempre de que la lógica sea correcta.

            TU TEMARIO DE EXPERTO:
            - Primeros pasos en Python
            - Tipos de datos, variables y operadores
            - Sentencias de control (Estructura condicional, repetitivas, anidadas)
            - Colecciones (Listas, Tuplas, Strings, Diccionarios, Integración colecciones)
            - Funciones
            - Archivos (Lectura/Escritura)

        3. [final_answer]: Cuando tengas la respuesta verificada para el usuario. El 'tool_input' debe ser la respuesta en lenguaje natural.

        REGLAS DE COMPORTAMIENTO (ANTI-LOOP):
        1. Si usaste una herramienta y obtuviste "✅ Ejecución Exitosa", NO la vuelvas a usar con el mismo código.
        2. Si ya tienes el resultado en el HISTORIAL, procede INMEDIATAMENTE a [final_answer].
        3. En [final_answer], muestra el código que generaste y explica la salida.

        HISTORIAL:
        {history}

        Responde SIEMPRE en formato JSON.
        Tu respuesta DEBE ser un objeto JSON que coincida exactamente con el siguiente esquema:
        ```json
        {schema}
        ```

        EJEMPLO DE RESPUESTA JSON:
        ```json
        {{
          "thought": "El usuario quiere la media de las notas de matemáticas. Usaré la herramienta 'pandas_analysis' para obtenerla.",
          "tool": "pandas_analysis",
          "tool_input": "df['math score'].mean()"
        }}
        ```
        """
        response = client.models.generate_content(
          model=self.model,
          contents=prompt,
          config={
              "response_mime_type": "application/json",
              "temperature": 0.0 # Bajar la temperatura para respuestas más determinísticas
          },
        )
        # Parseamos el JSON a nuestro objeto Pydantic
        return AgentAction.model_validate_json(response.text)

# --- 4. ORQUESTADOR (El Agente Principal) ---
class CortexAgent:
    def __init__(self, df):
        self.df = df
        self.brain = CortexBrain()
        self.tools = CortexTools(self.df)
        self.history = []
        self.logger = CortexLogger() # Instanciamos el Logger

    @weave.op()
    async def run(self, user_query: str):

        # 1. Visualización Inicial y Log
        console.rule("[bold cyan]Nueva Consulta[/]")
        console.print(f"👤 [bold]Usuario:[/bold] {user_query}")

        self.logger.log("INPUT USUARIO", user_query)
        self.history.append(f"Usuario: {user_query}")

        # Bucle de Agente (ReAct: Reason + Act)
        max_steps = 6
        for i in range(max_steps):
            console.print(f"\n🔄 [dim]Paso {i+1}...[/]")

            # CAMBIO 2: Throttling para gestión de cuota (Rate Limiting)
            # Esperamos 4 segundos. Esto limita a ~15 llamadas/minuto.
            # Si tienes una cuota más alta por ser Gemini 2.5, podrías bajarlo a 2s.
            await asyncio.sleep(4)

            # 1. PENSAR (LLM)

            try:
                action = await self.brain.decide(
                    history="\n".join(self.history),
                    schema=AgentAction.model_json_schema()
                )
            except Exception as e:
                error_msg = str(e)
                if "429" in error_msg:
                    msg_final = "⛔ ERROR CRÍTICO: Cuota Excedida (429). Deteniendo ejecución para evitar bloqueo."
                    console.print(Panel(msg_final, title="Error de Cuota", style="bold red"))
                    self.logger.log("ERROR_CUOTA", msg_final)
                    return msg_final # Cortamos aquí devolviendo el error
                else:
                    msg_tech = f"❌ Error técnico: {e}"
                    console.print(Panel(msg_tech, title="Error Desconocido", style="bold red"))
                    self.logger.log("ERROR_TECNICO", msg_tech)
                    # Si es otro error (ej: JSON mal formado), quizás sí quieras reintentar,
                    # pero por seguridad podemos cortar también.
                    return msg_tech

            # Log del Pensamiento (Archivo)
            self.logger.log("PENSAMIENTO", action.thought)
            # Visualización del Pensamiento (Panel Azul)
            console.print(Panel(action.thought, title=f"🧠 Pensamiento (Paso {i+1})", style="blue", border_style="blue"))

            # 2. ACTUAR (Ejecutar Tool)

            # --- VISUALIZACIÓN DE HERRAMIENTA (ACTUALIZADO) ---
            if action.tool != "final_answer":
                console.print(f"🟠 [bold orange3]Herramienta:[/bold orange3] {action.tool}")

                # Si es código, lo mostramos con resaltado de sintaxis
                if action.tool in ["pandas_analysis", "python_tutor_exec"]:
                    syntax = Syntax(action.tool_input, "python", theme="monokai", line_numbers=False)
                    console.print(syntax)
                else:
                    console.print(f"   Input: {action.tool_input}")

                self.logger.log("HERRAMIENTA_USO", f"{action.tool} -> {action.tool_input}")

            # --- EJECUCIÓN ESPECÍFICA ---

            if action.tool == "final_answer":
                # Renderizado Markdown en Panel Verde
                md_response = Markdown(action.tool_input)
                console.print(Panel(md_response, title="🤖 Respuesta Final", style="green", border_style="green"))

                self.logger.log("RESPUESTA_FINAL", action.tool_input)
                return action.tool_input

            elif action.tool == "pandas_analysis":
                observation = self.tools.pandas_analysis(action.tool_input)

                # Visualización discreta para datos
                console.print(f"📊 [dim]Observación: {observation}[/]")

                self.logger.log("OBSERVACION_PANDAS", observation)
                self.history.append(f"Agente: Usé ({action.tool}) con input '{action.tool_input}'. Resultado: {observation}")

            elif action.tool == "python_tutor_exec":
                console.print("💻 [dim]Ejecutando Script Python...[/]")

                if action.mock_inputs:
                    console.print(f"⌨️ [dim cyan]Inputs Simulados por el Agente: {action.mock_inputs}[/]")

                observation = self.tools.python_tutor_exec(action.tool_input, action.mock_inputs)

                # Visualización tipo Terminal para la salida de Python
                console.print(Panel(observation, title="📟 Salida de Consola", style="white", border_style="grey50"))

                self.logger.log("OBSERVACION_PYTHON", observation)
                self.history.append(f"Agente: Usé ({action.tool}): {observation}. TAREA COMPLETADA. VE A FINAL ANSWER.")

        msg_timeout = "No pude llegar a una conclusión en los pasos permitidos."
        console.print(f"[bold red]{msg_timeout}[/]")
        self.logger.log("TIMEOUT", msg_timeout)
        return msg_timeout

async def main():

    agent = CortexAgent(df)

    # Separador visual con Rich
    console.rule("[bold magenta]CASO 1: Data Science[/]")
    await agent.run("¿Podrías calcular la media, la mediana y la moda de las notas matemáticas (math score) de los alumnos y y decirme cuántos tienen asistencia (attendance) perfecta (1.0)?")

    console.rule("[bold magenta]CASO 2: Tutoría Python[/]")
    exercise_prompt = """
    Definir una función denominada crearArch() que le pida al usuario
    que ingrese nombres de estudiantes (ingresar solo 5 nombres como ejemplo)
    y los almacene en un archivo llamado estudiantes.txt (uno por línea).
    El ingreso de estudiantes debe finalizar cuando el usuario escriba "fin".
    """
    await agent.run(exercise_prompt)

if __name__ == "__main__":
    asyncio.run(main())

weave: Please login to Weights & Biases (https://wandb.ai) to continue...
wandb: WARNING The anonymous parameter to wandb.login() has no effect and will be removed in future versions.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=weave
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: leanai (teamlg-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
weave: wandb version 0.24.0 is available!  To upgrade, please run:
weave:  $ pip install wandb --upgrade
weave: Logged in as Weights & Biases user: leanai.
weave: View Weave data at https://wandb.ai/teamlg-ai/cortex-agent-multitool/weave


────────────────────────────────────────────── CASO 1: Data Science ───────────────────────────────────────────────

───────────────────────────────────────────────── Nueva Consulta ──────────────────────────────────────────────────

...

⌨️ Inputs Simulados por el Agente: ['Ana', 'Luis', 'Maria', 'Pedro', 'Sofia', 'fin']

╭───────────────────────────────────────────── 📟 Salida de Consola ──────────────────────────────────────────────╮
│ ✅ Ejecución Exitosa.                                                                                           │
│ Salida en consola:                                                                                              │
│ Ingresa nombres de estudiantes (escribe 'fin' para terminar):                                                   │
│ Ana                                                                                                             │
│ Luis                                                                                                            │
│ Maria                                                                                                           │
│ Pedro                                                                                                           │
│ Sofia                                                                                                           │
│ fin                                                                                                             │
│ Archivo 'estudiantes.txt' creado y nombres guardados exitosamente.                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔄 Paso 2...

╭──────────────────────────────────────────── 🧠 Pensamiento (Paso 2) ────────────────────────────────────────────╮
│ Ya he ejecutado el código para crear el archivo y guardar los nombres de los estudiantes. La ejecución fue      │
│ exitosa y la salida de la consola lo confirma. Ahora debo proporcionar la respuesta final al usuario,           │
│ incluyendo el código generado y una explicación.                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 🤖 Respuesta Final ───────────────────────────────────────────────╮
│ Aquí tienes la función crearArch() que solicita nombres de estudiantes y los guarda en un archivo               │
│ estudiantes.txt:                                                                                                │
│                                                                                                                 │
│                                                                                                                 │
│  def crearArch():                                                                                               │
│      with open('estudiantes.txt', 'w') as file:                                                                 │
│          print("Ingresa nombres de estudiantes (escribe 'fin' para terminar):")                                 │
│          nombres_ingresados = []                                                                                │
│          while True:                                                                                            │
│              nombre = input()                                                                                   │
│              if nombre.lower() == 'fin':                                                                        │
│                  break                                                                                          │
│              nombres_ingresados.append(nombre)                                                                  │
│              file.write(nombre + '\n')                                                                          │
│      print("Archivo 'estudiantes.txt' creado y nombres guardados exitosamente.")                                │
│                                                                                                                 │
│  crearArch()                                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
│ Explicación del código:                                                                                         │
│                                                                                                                 │
│   1 def crearArch():: Define una función llamada crearArch.                                                     │
│   2 with open('estudiantes.txt', 'w') as file:: Abre un archivo llamado estudiantes.txt en modo de escritura    │
│     ('w'). El uso de with asegura que el archivo se cierre automáticamente después de que se complete el bloque │
│     de código, incluso si ocurre un error.                                                                      │
│   3 print("Ingresa nombres de estudiantes (escribe 'fin' para terminar):"): Muestra una instrucción al usuario. │
│   4 nombres_ingresados = []: Inicializa una lista para almacenar temporalmente los nombres (aunque en este caso │
│     se escriben directamente al archivo, esta línea podría ser útil para otras lógicas).                        │
│   5 while True:: Inicia un bucle infinito para pedir nombres continuamente.                                     │
│   6 nombre = input(): Solicita al usuario que ingrese un nombre.                                                │
│   7 if nombre.lower() == 'fin': break: Si el usuario escribe "fin" (ignorando mayúsculas/minúsculas), el bucle  │
│     se rompe.                                                                                                   │
│   8 nombres_ingresados.append(nombre): Agrega el nombre ingresado a la lista nombres_ingresados.                │
│   9 file.write(nombre + '\n'): Escribe el nombre en el 

# Weave

Comparación de la implementación del Agente

Traces: https://wandb.ai/teamlg-ai/cortex-agent-multitool/weave/traces?view=traces_default

# Recursos
* [API de Gemini](https://ai.google.dev/api?hl=es-419)
* [Pydantic](https://docs.pydantic.dev/latest/)
* [Weave](https://docs.wandb.ai/weave)